In [1]:
!pip list | grep datasets
!pip list | grep sentencepiece
!pip list | grep transformers
!pip list | grep tqdm

datasets                      2.11.0
sentencepiece                 0.1.96
transformers                  4.28.1
tqdm                          4.65.0


In [2]:
# !pip install bitsandbytes
# !pip install loralib
# !pip install -q git+https://github.com/huggingface/peft.git
# !pip install --upgrade accelerate
# !pip install -U tqdm

In [3]:
import os

import torch
from torch import nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers as T
from transformers import AutoTokenizer, AutoConfig
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, get_peft_model_state_dict

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so...


/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so.

In [4]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3 
LEARNING_RATE = 3e-4
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 2000

In [5]:
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [7]:
model = prepare_model_for_int8_training(model)

In [8]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0

In [9]:
data = load_dataset("json", data_files="japanese_alpaca_data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-88fb7e0d1f58050f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]

In [12]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""
    
def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [13]:
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))

Map:   0%|          | 0/50002 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
world_size = int(os.environ.get('WORLD_SIZE', 1))
ddp = world_size != 1

In [15]:
ddp

False

In [16]:
trainer = T.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=T.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        save_steps=200,
        output_dir="../outputs/japanese-lora-alpaca_train_1",
        save_total_limit=3,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    data_collator=T.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.8/site-packages/transformers/utils/import_utils.py:1146 in _get_module    │
│                                                                                                  │
│   1143 │                                                                                         │
│   1144 │   def _get_module(self, module_name: str):                                              │
│   1145 │   │   try:                                                                              │
│ ❱ 1146 │   │   │   return importlib.import_module("." + module_name, self.__name__)              │
│   1147 │   │   except Exception as e:                                                            │
│   1148 │   │   │   raise RuntimeError(                                                           │
│   1149 │   │   │   │   f"Failed to import {self.__name__}.{module_name} because of the followin  │
│                                                                                                  │
│ /opt/conda/lib/python3.8/importlib/__init__.py:127 in import_module                              │
│                                                                                                  │
│   124 │   │   │   if character != '.':                                                           │
│   125 │   │   │   │   break                                                                      │
│   126 │   │   │   level += 1                                                                     │
│ ❱ 127 │   return _bootstrap._gcd_import(name[level:], package, level)                            │
│   128                                                                                            │
│   129                                                                                            │
│   130 _RELOADING = {}                                                                            │
│ in _gcd_import                                                                                   │
│ in _find_and_load                                                                                │
│ in _find_and_load_unlocked                                                                       │
│ in _load_unlocked                                                                                │
│ in exec_module                                                                                   │
│ in _call_with_frames_removed                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.8/site-packages/transformers/trainer.py:173 in <module>                   │
│                                                                                                  │
│    170 │   DEFAULT_PROGRESS_CALLBACK = NotebookProgressCallback                                  │
│    171                                                                                           │
│    172 if is_apex_available():                                                                   │
│ ❱  173 │   from apex import amp                                                                  │
│    174                                                                                           │
│    175 if is_datasets_available():                                                               │
│    176 │   import datasets                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'amp' from 'apex' (unknown location)

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>   

In [20]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/home/fnakamura/workspace/apex
Using pip 23.1.2 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /home/fnakamura/workspace/apex
  Running command python setup.py egg_info


  torch.__version__  = 1.13.1+cu117


  running egg_info
  creating /tmp/pip-pip-egg-info-h85js14d/apex.egg-info
  writing /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-h85js14d/apex.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-h85js14d/apex.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-